In [ ]:
!conda install numpy scipy scikit-learn pandas joblib
!pip install deap update_checker tqdm stopit
!pip install dask[delayed] dask-ml
!pip install scikit-mdr skrebate
!pip install tpot
!conda install -c conda-forge ipywidgets


In [28]:
import pandas as pd
import numpy as np
import os
import random
random.seed(100)

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

In [29]:
#from google.colab import drive
#drive.mount('/content/drive')

In [30]:
#os.chdir('/home/')
train = pd.read_csv('exoTrain.csv')

In [31]:
train.shape

(5087, 3198)

In [32]:
train.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [33]:
train.rename(columns={'LABEL': 'class'}, inplace=True)

In [34]:
#train.dtypes

In [35]:
train.isnull().values.any()

False

In [36]:
print(train.shape)
train=train.dropna()
print(train.shape)

(5087, 3198)
(5087, 3198)


In [37]:
train.isnull().values.any()

False

In [38]:
for col_name in train.columns:
    if(train[col_name].dtype == 'object'):
        print(col_name)
        train[col_name]= train[col_name].astype('category')
        train[col_name] = train[col_name].cat.codes

In [39]:
train.head()

,class,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [40]:
X = train.drop('class', axis=1)  
#X = X.drop('Loan_ID',axis=1)
y = train['class'] 

In [41]:
X.head()

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,-160.17,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,-73.38,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,484.39,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,323.33,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,-970.88,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [42]:
y.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: int64

In [44]:

X_train, X_test, y_train, y_test = train_test_split(X, y.values,train_size=0.90, test_size=0.10)

In [45]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train_array=X_train.values
X_test_array=X_test.values
print(type(X_train_array),type(X_test_array), type(y_train), np.shape(X_train_array), np.shape(X_test_array))

(4578, 3197) (509, 3197) (4578,) (509,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> (4578, 3197) (509, 3197)


In [46]:
print(X_train_array[1:4])
print(y_train[1:4])
X_train[1:4]


[[   6.64   -0.97    3.36 ...   12.94   34.68   25.63]
 [  28.71   23.88   14.8  ...   -4.78   11.23    3.46]
 [ 323.51  285.75  283.15 ... -130.96 -101.02  -85.14]]
[1 1 1]


,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
1949,6.64,-0.97,3.36,-20.28,25.21,14.05,11.92,5.28,2.18,11.89,...,77.05,105.71,131.03,189.64,219.13,-6.37,-6.37,12.94,34.68,25.63
1893,28.71,23.88,14.80,12.61,16.86,17.06,15.61,20.79,11.97,6.54,...,10.93,19.32,7.18,11.95,4.94,1.28,2.97,-4.78,11.23,3.46
4511,323.51,285.75,283.15,237.64,208.31,142.98,78.37,73.03,50.20,23.95,...,-93.96,-99.42,-32.63,-80.77,-29.91,-55.91,-139.32,-130.96,-101.02,-85.14


In [47]:
print(y_test[1:4])
print(X_test_array[1:4])
X_test[1:4]

[1 1 1]
[[-1549.05 -1576.43 -1553.18 ...   314.58   280.01   306.14]
 [    5.46    10.28     7.87 ...     2.25    -2.96     8.83]
 [    2.7     -6.37   -13.93 ...    -4.33    -6.72    -2.13]]


,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
509,-1549.05,-1576.43,-1553.18,-1544.52,-1539.99,-1561.30,-1537.07,-1566.56,-1570.26,-1560.57,...,642.26,646.83,570.76,478.92,364.31,126.94,296.15,314.58,280.01,306.14
4736,5.46,10.28,7.87,6.57,3.35,4.70,3.96,2.43,2.45,3.69,...,-2.45,-1.59,-3.60,-2.11,-7.06,0.72,-4.71,2.25,-2.96,8.83
2812,2.70,-6.37,-13.93,-5.82,-4.59,-1.29,-2.37,-0.92,4.70,-2.86,...,-1.91,-0.48,-3.75,6.65,-2.17,9.53,-4.82,-4.33,-6.72,-2.13


In [48]:
!pwd

/rapids/notebooks/utils/hostdir/capstone


In [49]:
!mkdir tpot_checkpoint

mkdir: cannot create directory 'tpot_checkpoint': File exists


In [52]:
#print(np.shape(X_train_array), np.shape(X_test_array))

exoplanet_model_tpot = TPOTClassifier(
                          generations = 5,    #100: Number of iterations to the run pipeline optimization process. #TPOT will evaluate population_size + generations × offspring_size pipelines in total. 
                          population_size=10, #100: Number of individuals to retain in the genetic programming population every generation. 
                          offspring_size = 2, #None: Number of offspring to produce in each genetic programming generation. By default, the number of offspring is equal to the number of population size. 
                          mutation_rate=0.8,  #0.9: Mutation rate for the genetic programming algorithm in the range [0.0, 1.0] to random changes to every generation to those many pipelines. mutation_rate + crossover_rate cannot exceed 1.0.
                          crossover_rate=0.2, #0.1: Crossover rate for the genetic programming algorithm in the range [0.0, 1.0] to breed those many pipelines every generation.  mutation_rate + crossover_rate cannot exceed 1.0. 
                          scoring='accuracy', #accuracy: One of the available scoring functions like 'accuracy', 'f1', 'precision', 'recall', 'roc_auc', etc
                          cv=5,               #5: Cross-validation strategy used when evaluating pipelines. An integer or a cross-validation generator object or an iterable yielding train/test splits
                          subsample = 1.0,    #1.0: Fraction of training samples that are used during the TPOT optimization process. Must be in the range (0.0, 1.0]. 
                          n_jobs = 2,         #1: Number of processes to use in parallel for evaluating pipelines during the TPOT optimization process. Setting n_jobs=-1 will use all cores, -2 will use all cores but one. Using more cores requires more memory.
                          max_time_mins=120,  #None: How many minutes TPOT has to optimize the pipeline.If not None, this setting will override the generations parameter and allow TPOT to run until max_time_mins minutes elapse. 
                          max_eval_time_mins=5,    #5: How many minutes TPOT has to evaluate a single pipeline. Use this parameter to help prevent TPOT from wasting time on evaluating time-consuming pipelines. 
                          random_state=3,     #None: Integer. Use this parameter to make sure that TPOT will give you the same results each time you run it against the same data set with that seed.
                          config_dict = None, #None: 'TPOT light', 'TPOT Spare', 'TPOT MDR', None or a python dictionary for custom config
                       #   template = None,    #None: Template of predefined pipeline structure. 
                          warm_start = True,  #False: Flag indicating whether the TPOT instance will reuse the population from previous calls to fit(). 
                          memory = 'auto',    #None: 'auto' or a caching dir  path or Memory object or None. 
                          use_dask = False,   #False: Whether to use Dask-ML's pipeline optimiziations. This avoid re-fitting the same estimator on the same split of data multiple times. It will also provide more detailed diagnostics when using Dask's distributed scheduler. 
                          periodic_checkpoint_folder = '/rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint', #None: If supplied, a folder in which TPOT will periodically save pipelines in pareto front so far while optimizing.
                          early_stop=None,    #None: Ends the optimization process if there is no improvement in the given number of generations. 
                          verbosity=3,        #0: 0 to 4.  How much information TPOT communicates while it's running. 
                       )



In [53]:
#tpot_exoplanet_3mins = TPOTClassifier(verbosity=2, max_time_mins=60)
#tpot_exoplanet_3mins.fit(X_train_array, y_train)

exoplanet_model_tpot.fit(X_train_array, y_train)

30 operators have been imported by TPOT.


Skipped pipeline #1 due to time out. Continuing to the next pipeline.
Skipped pipeline #3 due to time out. Continuing to the next pipeline.
Skipped pipeline #9 due to time out. Continuing to the next pipeline.
Saving periodic pipeline from pareto front to /rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint/pipeline_gen_1_idx_0_2019.09.09_12-01-43.py
Generation 1 - Current Pareto front scores:
-1	0.9925735525976321	GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.001, GradientBoostingClassifier__max_depth=5, GradientBoostingClassifier__max_features=0.2, GradientBoostingClassifier__min_samples_leaf=12, GradientBoostingClassifier__min_samples_split=19, GradientBoostingClassifier__n_estimators=100, GradientBoostingClassifier__subsample=1.0)

Periodic pipeline was not saved, probably saved before...
Generation 2 - Current Pareto front scores:
-1	0.9925735525976321	GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.00

Skipped pipeline #50 due to time out. Continuing to the next pipeline.
Skipped pipeline #52 due to time out. Continuing to the next pipeline.
Generation 16 - Current Pareto front scores:
-1	0.9925735525976321	GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.001, GradientBoostingClassifier__max_depth=5, GradientBoostingClassifier__max_features=0.2, GradientBoostingClassifier__min_samples_leaf=12, GradientBoostingClassifier__min_samples_split=19, GradientBoostingClassifier__n_estimators=100, GradientBoostingClassifier__subsample=1.0)

Periodic pipeline was not saved, probably saved before...
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 17 - Current Pareto front scores:
-1	0.9925735525976321	GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.001, GradientBoostingClassifier__max_depth=5, GradientBoostingClassifier__max_fe

TPOTClassifier(config_dict=None, crossover_rate=0.2, cv=5,
               disable_update_check=False, early_stop=None, generations=1000000,
               max_eval_time_mins=5, max_time_mins=120, memory='auto',
               mutation_rate=0.8, n_jobs=2, offspring_size=2,
               periodic_checkpoint_folder='/rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint',
               population_size=10, random_state=3, scoring='accuracy',
               subsample=1.0, template='RandomTree', use_dask=False,
               verbosity=3, warm_start=True)

In [70]:
y_pred=exoplanet_model_tpot.predict(X_test_array)
y_train_pred=exoplanet_model_tpot.predict(X_train_array)

In [54]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [68]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [69]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('\n clasification report:\n', classification_report(y_test,y_pred))
print('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))

Accuracy: 0.9941060903732809
F1 score: 0.9970443349753695
Recall: 1.0
Precision: 0.9941060903732809

 clasification report:
               precision    recall  f1-score   support

           1       0.99      1.00      1.00       506
           2       0.00      0.00      0.00         3

    accuracy                           0.99       509
   macro avg       0.50      0.50      0.50       509
weighted avg       0.99      0.99      0.99       509


 confussion matrix:
 [[506   0]
 [  3   0]]


/conda/envs/rapids/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
print(exoplanet_model_tpot.score(X_test_array, y_test))

0.9941060903732809


In [71]:
print('Accuracy:', accuracy_score(y_train, y_train_pred))
print('F1 score:', f1_score(y_train, y_train_pred))
print('Recall:', recall_score(y_train, y_train_pred))
print('Precision:', precision_score(y_train, y_train_pred))
print('\n clasification report:\n', classification_report(y_train, y_train_pred))
print('\n confussion matrix:\n',confusion_matrix(y_train, y_train_pred))

Accuracy: 0.9925731760594146
F1 score: 0.9962727472045604
Recall: 1.0
Precision: 0.9925731760594146

 clasification report:
               precision    recall  f1-score   support

           1       0.99      1.00      1.00      4544
           2       0.00      0.00      0.00        34

    accuracy                           0.99      4578
   macro avg       0.50      0.50      0.50      4578
weighted avg       0.99      0.99      0.99      4578


 confussion matrix:
 [[4544    0]
 [  34    0]]


/conda/envs/rapids/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:
#os.chdir('/content/drive/My Drive/Colab Notebooks/AUTOML')
#!head exotrain_processed.csv
!ls -lrt exotrain_processed.csv

-rw-r--r-- 1 root root 271030495 Sep  9 14:19 exotrain_processed.csv


In [72]:

exoplanet_model_tpot.export('exoplanet_model_tpot.py')

#tpot_model2.export('tpot_loanpred_model2_30mins.py')

!cat exoplanet_model_tpot.py




import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=3)

# Average CV score on the training set was:0.9925735525976321
exported_pipeline = GradientBoostingClassifier(learning_rate=0.001, max_depth=5, max_features=0.2, min_samples_leaf=12, min_samples_split=19, n_estimators=100, subsample=1.0)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [74]:
import pickle 
#exoplanet_model_tpot_pickle = pickle.dumps(exoplanet_model_tpot.fitted_pipeline_)
#exoplanet_model_tpot1 = pickle.loads(tpot_exoplanet_model_pickle)
# save the model to disk
pickle.dump(exoplanet_model_tpot.fitted_pipeline_, open('exoplanet_model_tpot_pickle.pkl', 'wb'))

# load the model from disk
exoplanet_model_tpot1 = pickle.load(open('exoplanet_model_tpot_pickle.pkl', 'rb'))

!ls -lrt exoplanet_model_tpot_pickle.pkl
print(exoplanet_model_tpot.score(X_test_array, y_test))


-rw-r--r-- 1 root root 307584 Sep  9 14:45 exoplanet_model_tpot_pickle.pkl
0.9941060903732809


In [76]:
from joblib import dump, load
dump(exoplanet_model_tpot.fitted_pipeline_, 'exoplanet_model_tpot.joblib')
exoplanet_model_tpot2 = load('exoplanet_model_tpot.joblib')
print(exoplanet_model_tpot2.score(X_test_array, y_test))
!ls -lrt exoplanet_model_tpot.joblib

0.9941060903732809
-rw-r--r-- 1 root root 412302 Sep  9 14:47 exoplanet_model_tpot.joblib


In [77]:
!mkdir /rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint2

In [80]:
exoplanet_model_tpot_f1 = TPOTClassifier(
                          generations = 5,    #100: Number of iterations to the run pipeline optimization process. #TPOT will evaluate population_size + generations × offspring_size pipelines in total. 
                          population_size=10, #100: Number of individuals to retain in the genetic programming population every generation. 
                          offspring_size = 2, #None: Number of offspring to produce in each genetic programming generation. By default, the number of offspring is equal to the number of population size. 
                          mutation_rate=0.8,  #0.9: Mutation rate for the genetic programming algorithm in the range [0.0, 1.0] to random changes to every generation to those many pipelines. mutation_rate + crossover_rate cannot exceed 1.0.
                          crossover_rate=0.2, #0.1: Crossover rate for the genetic programming algorithm in the range [0.0, 1.0] to breed those many pipelines every generation.  mutation_rate + crossover_rate cannot exceed 1.0. 
                          scoring='f1_weighted', #accuracy: One of the available scoring functions like 'accuracy', 'f1', 'precision', 'recall', 'roc_auc', etc
                          cv=5,               #5: Cross-validation strategy used when evaluating pipelines. An integer or a cross-validation generator object or an iterable yielding train/test splits
                          subsample = 1.0,    #1.0: Fraction of training samples that are used during the TPOT optimization process. Must be in the range (0.0, 1.0]. 
                          n_jobs = 4,         #1: Number of processes to use in parallel for evaluating pipelines during the TPOT optimization process. Setting n_jobs=-1 will use all cores, -2 will use all cores but one. Using more cores requires more memory.
                          max_time_mins=120,  #None: How many minutes TPOT has to optimize the pipeline.If not None, this setting will override the generations parameter and allow TPOT to run until max_time_mins minutes elapse. 
                          max_eval_time_mins=10,    #5: How many minutes TPOT has to evaluate a single pipeline. Use this parameter to help prevent TPOT from wasting time on evaluating time-consuming pipelines. 
                          random_state=3,     #None: Integer. Use this parameter to make sure that TPOT will give you the same results each time you run it against the same data set with that seed.
                          config_dict = 'TPOT light', #None: 'TPOT light', 'TPOT Spare', 'TPOT MDR', None or a python dictionary for custom config
                       #   template = None,    #None: Template of predefined pipeline structure. 
                          warm_start = False,  #False: Flag indicating whether the TPOT instance will reuse the population from previous calls to fit(). 
                          memory = 'auto',    #None: 'auto' or a caching dir  path or Memory object or None. 
                          use_dask = False,   #False: Whether to use Dask-ML's pipeline optimiziations. This avoid re-fitting the same estimator on the same split of data multiple times. It will also provide more detailed diagnostics when using Dask's distributed scheduler. 
                          periodic_checkpoint_folder = '/rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint2', #None: If supplied, a folder in which TPOT will periodically save pipelines in pareto front so far while optimizing.
                          early_stop=None,    #None: Ends the optimization process if there is no improvement in the given number of generations. 
                          verbosity=3,        #0: 0 to 4.  How much information TPOT communicates while it's running. 
                       )

In [81]:
exoplanet_model_tpot_f1.fit(X_train_array, y_train)

19 operators have been imported by TPOT.


Skipped pipeline #1 due to time out. Continuing to the next pipeline.
Skipped pipeline #4 due to time out. Continuing to the next pipeline.
Saving periodic pipeline from pareto front to /rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint2/pipeline_gen_1_idx_0_2019.09.09_15-09-42.py
Generation 1 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)

Skipped pipeline #15 due to time out. Continuing to the next pipeline.
Generation 2 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)

Periodic pipeline was not saved, probably saved before...
Generation 3 - Current Pareto fro

Generation 13 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)
-3	0.9893647051551483	DecisionTreeClassifier(BernoulliNB(RobustScaler(input_matrix), BernoulliNB__alpha=100.0, BernoulliNB__fit_prior=False), DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=2, DecisionTreeClassifier__min_samples_leaf=18, DecisionTreeClassifier__min_samples_split=3)

Periodic pipeline was not saved, probably saved before...
Periodic pipeline was not saved, probably saved before...
Generation 14 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)
-3	0.9893647051551483	De

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='logistic_regression' are not supported when dual=True, Parameters: penalty='l1', loss='logistic_regression', dual=True.
Generation 25 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)
-3	0.9893647051551483	DecisionTreeClassifier(BernoulliNB(RobustScaler(input_matrix), BernoulliNB__alpha=100.0, BernoulliNB__fit_prior=False), DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=2, DecisionTreeClassifier__min_samples_leaf=18, DecisionTreeClassifier__min_samples_split=3)

Skipped pipeline #66 due to time out. Continuing to the next pipeline.
Generation 26 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(inp

Generation 37 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)
-3	0.9893647051551483	DecisionTreeClassifier(BernoulliNB(RobustScaler(input_matrix), BernoulliNB__alpha=100.0, BernoulliNB__fit_prior=False), DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=2, DecisionTreeClassifier__min_samples_leaf=18, DecisionTreeClassifier__min_samples_split=3)

Skipped pipeline #90 due to time out. Continuing to the next pipeline.
Generation 38 - Current Pareto front scores:
-1	0.9890767724740954	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=4, DecisionTreeClassifier__min_samples_leaf=11, DecisionTreeClassifier__min_samples_split=13)
-3	0.9893647051551483	DecisionTreeClassifier(BernoulliNB(RobustScaler

TPOTClassifier(config_dict='TPOT light', crossover_rate=0.2, cv=5,
               disable_update_check=False, early_stop=None, generations=1000000,
               max_eval_time_mins=10, max_time_mins=120, memory='auto',
               mutation_rate=0.8, n_jobs=4, offspring_size=2,
               periodic_checkpoint_folder='/rapids/notebooks/utils/hostdir/capstone/tpot_checkpoint2',
               population_size=10, random_state=3, scoring='f1_weighted',
               subsample=1.0, template='RandomTree', use_dask=False,
               verbosity=3, warm_start=False)

In [83]:
import pickle 
#exoplanet_model_tpot_pickle = pickle.dumps(exoplanet_model_tpot.fitted_pipeline_)
#exoplanet_model_tpot1 = pickle.loads(tpot_exoplanet_model_pickle)
# save the model to disk
pickle.dump(exoplanet_model_tpot_f1.fitted_pipeline_, open('exoplanet_model_tpot_pickle_f1.pkl', 'wb'))

# load the model from disk
exoplanet_model_tpot1_f1_load = pickle.load(open('exoplanet_model_tpot_pickle_f1.pkl', 'rb'))

!ls -lrt exoplanet_model_tpot_pickle.pkl
print(exoplanet_model_tpot1_f1_load.score(X_test_array, y_test))

-rw-r--r-- 1 root root 307584 Sep  9 14:45 exoplanet_model_tpot_pickle.pkl
0.9941060903732809


In [84]:
y_pred2=exoplanet_model_tpot1_f1_load.predict(X_test_array)
y_train_pred2=exoplanet_model_tpot1_f1_load.predict(X_train_array)
print('Accuracy:', accuracy_score(y_train, y_train_pred2))
print('F1 score:', f1_score(y_train, y_train_pred2))
print('Recall:', recall_score(y_train, y_train_pred2))
print('Precision:', precision_score(y_train, y_train_pred2))
print('\n clasification report:\n', classification_report(y_train, y_train_pred2))
print('\n confussion matrix:\n',confusion_matrix(y_train, y_train_pred2))

print('Accuracy:', accuracy_score(y_test, y_pred2))
print('F1 score:', f1_score(y_test, y_pred2))
print('Recall:', recall_score(y_test, y_pred2))
print('Precision:', precision_score(y_test, y_pred2))
print('\n clasification report:\n', classification_report(y_test,y_pred2))
print('\n confussion matrix:\n',confusion_matrix(y_test, y_pred2))



Accuracy: 0.9925731760594146
F1 score: 0.9962727472045604
Recall: 1.0
Precision: 0.9925731760594146

 clasification report:
               precision    recall  f1-score   support

           1       0.99      1.00      1.00      4544
           2       0.00      0.00      0.00        34

    accuracy                           0.99      4578
   macro avg       0.50      0.50      0.50      4578
weighted avg       0.99      0.99      0.99      4578


 confussion matrix:
 [[4544    0]
 [  34    0]]
Accuracy: 0.9941060903732809
F1 score: 0.9970443349753695
Recall: 1.0
Precision: 0.9941060903732809

 clasification report:
               precision    recall  f1-score   support

           1       0.99      1.00      1.00       506
           2       0.00      0.00      0.00         3

    accuracy                           0.99       509
   macro avg       0.50      0.50      0.50       509
weighted avg       0.99      0.99      0.99       509


 confussion matrix:
 [[506   0]
 [  3   0]]


/conda/envs/rapids/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
